# [Advent of Code 2018](https://adventofcode.com/2018)

A collection of common functions to be used across problems, will add more as a generic use case for each comes up!

In [1]:
import heapq
import os
import re
from collections import Counter, defaultdict, namedtuple
from datetime import datetime
from itertools import cycle
from string import ascii_uppercase


cat = ''.join


Point = namedtuple('Point', 'x,y')


def Input(day):
    """Fetch the data input from disk."""
    filename = os.path.join('../data/advent2018/input{}.txt'.format(day))
    return open(filename)


def hamming_distance(s1, s2):
    """Number of non equal characters between two strings."""
    assert len(s1) == len(s2), 'Strings are not equal length'
    return sum(
        char1 != char2
        for char1, char2
        in zip(s1, s2)
    )


def chunks(l, n):
    """Yield successive n-sized chunks from l."""
    for i in range(0, len(l), n):
        yield l[i:i + n]
        

def manhattan_distance(p1, p2):
    return sum(
        abs(a - b) for a, b in zip(p1, p2)
    )


# Trees

class Node(object):
    def __init__(self, name):
        self.name = name
        self._children = []
        self.parent = None
        
    def __repr__(self):
        return '<Node 0x:{} name={}>'.format(
            id(self),
            self.name
        )
    
    @property
    def children(self):
        return tuple(self._children)
    
    @children.setter
    def children(self, value):
        for val in value:
            self.add_child(val)
    
    def add_child(self, child):
        self._children.append(child)
    
    def remove_child(self, child):
        self._children.remove(child)
    
    def add_parent(self, parent):
        if self.parent:
            self.parent.remove_child(self)

        self.parent = parent

        if self.parent:
            self.parent.add_child(self)
    
    @property
    def root(self):
        if not self.parent:
            return self
        return self.parent.root
    
    @property
    def decendants(self):
        iterator = visit_pre_order(self)
        # Skip this node
        next(iterator)
        return iterator
    
    @property
    def siblings(self):
        for child in self.parent.children:
            if child == self:
                continue
            yield child
            
    @property
    def ancestors(self):
        node = self
        while node.parent:
            yield node.parent
            node = node.parent
            
    @property
    def root(self):
        root = None
        for parent in self.ancestors:
            root = parent
        return root
    
    @property
    def height(self):
        return len(list(self.ancestors))


def visit_pre_order(node):
    """Visit a Tree in pre-order.
    
      A
     / \
    B   C
    
    A -> B -> C
    """
    yield node
    for child in node.children:
        yield from visit_pre_order(child)

## [Day 1: Chronal Calibration](https://adventofcode.com/2018/day/1)

The first part simply requires us to sum the values in a list to apply all the changes from the given delas.

In [2]:
def parse_input(initial_data):
    res = []
    for data in initial_data.readlines():
        res.append(int(data))
    return res

data = parse_input(Input(1))

In [3]:
sum(data)

525

Second portion requires us to find the first repeated value if we continually sum items in the delta list. A set is ideal here as it's O(1) for membership tests.

In [4]:
def find_first_repeat(deltas):
    seen = set([0])
    position = 0
    
    for delta in cycle(deltas):
        position += delta
        if position in seen:
            break
        seen.add(position)
    return position

assert find_first_repeat([7, 7, -2, -7, -4]) == 14

In [5]:
find_first_repeat(data)

75749

# [Day 2: Inventory Management System](https://adventofcode.com/2018/day/2)

In [6]:
data = [line.strip() for line in Input(2).readlines()]

In [7]:
from collections import Counter

def calculate_checksum(data):
    num_threes = 0
    num_twos = 0
    for line in data:
        counts = Counter(line)
        if 3 in counts.values():
            num_threes += 1
        if 2 in counts.values():
            num_twos += 1
        
    return num_threes * num_twos


test_data = [
    "abcdef",
    "bababc",
    "abbcde",
    "abcccd",
    "aabcdd",
    "abcdee",
    "ababab",
]
assert calculate_checksum(test_data) == 12
calculate_checksum(data)
    

6944

We're told that we need to find two strings that have only one character difference between them, this is also known as the the [Hamming Distance](https://en.wikipedia.org/wiki/Hamming_distance). So we're looking for two strings for which the hamming distance bewteen them is one, simple enough!

In [8]:
def find_boxes(data):
    for index, row in enumerate(data[:-1]):
        for comparison in data[index:]:
            if hamming_distance(row, comparison) == 1:
                same_chars = [
                    char1
                    for (char1, char2) in zip(row, comparison)
                    if char1 == char2
                ]
                return ''.join(same_chars)

find_boxes(data)

'srijafjzloguvlntqmphenbkd'

## [Day 3: No Matter How You Slice It](https://adventofcode.com/2018/day/3)

In [9]:
def parse_input(data):
    reg = re.compile('#(\d+) @ (\d+),(\d+): (\d+)x(\d+)')
    datum = namedtuple('datum', 'claim,left,top,width,height')
    parsed = []
    for line in data:
        match = reg.match(line)
        
        parsed.append(
            datum(*[int(i) for i in match.groups()])
        )
    return parsed


def find_overlaps(data):
    overlaps = defaultdict(list)
    repeated = 0
    
    for datum in data:
        for x in range(datum.width):
            for y in range(datum.height):
                p = Point(datum.left + x, datum.top + y)
                overlaps[p].append(datum.claim)
    return overlaps


def count_overlaps(data):
    overlaps = find_overlaps(data)
    return len([
        claims for claims in overlaps.values()
        if len(claims) > 1
    ])
    
        
test_data = [
    '#1 @ 1,3: 4x4',
    '#2 @ 3,1: 4x4',
    '#3 @ 5,5: 2x2',
]
test_data = parse_input(test_data)
assert count_overlaps(test_data) == 4

data = parse_input(Input(3).readlines())
count_overlaps(data)

110827

In [10]:
def find_unique(data):
    remainin_claims = set(
        datum.claim for datum in data
    )
    overlaps = find_overlaps(data)
    
    for claims in overlaps.values():
        if len(claims) == 1:
            continue
        remainin_claims -= set(claims)
    assert len(list(remainin_claims)) == 1
    return list(remainin_claims)[0]


assert find_unique(test_data) == 3
find_unique(data)

116

## [Day 4: Repose Record](https://adventofcode.com/2018/day/4)

The most fiddly portion of this problem was just parsing the data! The incoming events log are stateful, in that the current line applies to the most recently seen guard id.

In [11]:
Event = namedtuple('Event', 'asleep,wake_up')


def parse_data(lines):
    """
        Sort random order events before grouping them into
        (alseep, wake_up) datetime pairs.
    """
    data = []
    
    for line in lines:
        date_str = line[1:17]
        event = line[19:]
        data.append(
            (
                datetime.strptime(date_str, '%Y-%m-%d %H:%M'),
                event
            )
        )
    data = sorted(data)
    
    guard_events = defaultdict(list)

    current_guard = None
    data_iter = iter(data)
    try:
        while True:
            event = next(data_iter)
            guard_number = re.match('Guard #(\d+)', event[1])
            
            if guard_number:
                current_guard = int(guard_number.groups()[0])
                asleep = next(data_iter)
            else:
                asleep = event

            wakes_up = next(data_iter)

            guard_events[current_guard].append(
                Event(
                    asleep[0],
                    wakes_up[0]
                )
            )

    except StopIteration:
        pass        
    return guard_events

Phew, now that that's done we can get on with the problem!

All we need to do here is find the guard that's spent the most time asleep, the multiple the time he's most likely to be asleep (i.e the most observed sleepiest minute) and multiple that by the guard's id.

In [12]:
def minutes_asleep(guard_events):
    sleep_counter = defaultdict(int)
    
    for guard, events in guard_events.items():
        for asleep, wake_up in events:
            sleep_counter[guard] += wake_up.minute - asleep.minute
            
    return sleep_counter


def most_common_time_asleep(sleepiest_events):
    min_counter = Counter()
    for asleep, awake in sleepiest_events:
        min_counter.update(
            range(asleep.minute, awake.minute)
        )
    return min_counter.most_common(1)[0][0]

    
def strat_1(guard_events):
    sleep_counter = minutes_asleep(guard_events)

    sleepiest_guard = None
    max_time_asleep = 0
    for guard, time_asleep in sleep_counter.items():
        if time_asleep > max_time_asleep:
            max_time_asleep = time_asleep
            sleepiest_guard = guard
    
    common_minute = most_common_time_asleep(
        guard_events[sleepiest_guard]
    )
    return sleepiest_guard * common_minute

    

test_data = [
    '[1518-11-01 00:00] Guard #10 begins shift',
    '[1518-11-01 00:05] falls asleep',
    '[1518-11-01 00:25] wakes up',
    '[1518-11-01 00:30] falls asleep',
    '[1518-11-01 00:55] wakes up',
    '[1518-11-01 23:58] Guard #99 begins shift',
    '[1518-11-02 00:40] falls asleep',
    '[1518-11-02 00:50] wakes up',
    '[1518-11-03 00:05] Guard #10 begins shift',
    '[1518-11-03 00:24] falls asleep',
    '[1518-11-03 00:29] wakes up',
    '[1518-11-04 00:02] Guard #99 begins shift',
    '[1518-11-04 00:36] falls asleep',
    '[1518-11-04 00:46] wakes up',
    '[1518-11-05 00:03] Guard #99 begins shift',
    '[1518-11-05 00:45] falls asleep',
    '[1518-11-05 00:55] wakes up',
]

test_data = parse_data(test_data)
assert strat_1(test_data) == 240
data = parse_data(Input(4).readlines())
strat_1(data)

101262

In part two we're required to find the most common time to be asleep across all guards, and multuple that by the guard responsible for being asleep at that time.

In [13]:
def strat_2(guard_events):
    guard_counters = defaultdict(Counter)
    for guard, events in guard_events.items():
        for asleep, awake in events:
            guard_counters[guard].update(
                range(asleep.minute, awake.minute)
            )

    most_occurances = 0
    most_occurances_minute = 0
    most_occurances_guard = 0
    for guard, counter in guard_counters.items():
        most_common_min, occurances = counter.most_common(1)[0]
        if occurances > most_occurances:
            most_occurances = occurances
            most_occurances_minute = most_common_min
            most_occurances_guard = guard
            
    return most_occurances_guard * most_occurances_minute
            
    
assert strat_2(test_data) == 4455
strat_2(data)

71976

## [Day 5: Alchemical Reduction](https://adventofcode.com/2018/day/5)

This day specifically came with a warning about the size of the input data, so I downloaded it directly. This contained a new line which I did not strip, so got the wrong result for quite a while!

(This was rewritten to a more elegant solution thanks to [jumpertown's](https://github.com/jumpertown/aoc/blob/master/2018/Day_5_Alchemical_Reduction.ipynb) work! Much, much nicer, and an improved learning experience for me!

In [14]:
def does_react(c1, c2):
    return c1 != c2 and c1.lower() == c2.lower()


def reduce_polymer(data, remove_unit=''):
    if remove_unit:
        data = data.replace(remove_unit.lower(), '')
        data = data.replace(remove_unit.upper(), '')

    reduced = []

    for char in data:
        if reduced and does_react(char, reduced[-1]):
            reduced.pop()
        else:
            reduced.append(char)
    
    return cat(reduced)



test_input = 'dabAcCaCBAcCcaDA'
assert reduce_polymer(test_input) == 'dabCBAcaDA'

data = Input(5).read()
len(reduce_polymer(data))

9462

In [15]:
def ultimate_reduction(data):
    chars = set(data.lower())
    shortest = len(data)
    for char in chars:
        reduced = reduce_polymer(data, char)
        if len(reduced) < shortest:
            shortest = len(reduced)
    return shortest


assert ultimate_reduction(test_input) == 4
ultimate_reduction(data)

4952

## [Day 6: Chronal Coordinates](https://adventofcode.com/2018/day/6)

Annoyingly this day had a bug in the solution submission, which affected enough people for the scoreboard contributions to be cleared on this day - not that I would have ranked of course, but still infuriating when your rather simple solution is somehow incorrect...

In [16]:
def parse_input(data):
    points = []
    for line in data.strip().split('\n'):
        x, y = line.split(', ')
        points.append(
            Point(int(x), int(y))
        )
        
    return points
        

def largest_area(points):
    max_x = max(p.x for p in points)
    max_y = max(p.y for p in points)
    
    # First assign each point to it's closest defined pooint
    assignments = defaultdict(list)
    for x in range(0, max_x + 1):
        for y in range(0, max_y + 1):
            p2 = Point(x, y)
            smallest = None
            winner = None
            
            for p1 in points:
                distance = manhattan_distance(p1, p2)                    
                if distance == smallest:
                    winner = None

                if smallest is None or distance < smallest:
                    smallest = distance
                    winner = p1
            if winner:
                assignments[winner].append(p2)
    
    # Now find any finite areas that are present
    finite_points = []
    for p in points:
        for p2 in assignments[p]:
            if (p2.x == 0 or p2.x == max_x or
                p2.y == 0 or p2.y == max_y):
                # has an edge, and is therefore infinite.
                break
        else:
            finite_points.append(p)
    
    # Of those, find which has the largest area
    overall_max = 0
    for p in finite_points:
        closest_points = assignments[p]
        if len(closest_points) > overall_max:
            overall_max = len(closest_points)
    return overall_max
        


test_data = """
1, 1
1, 6
8, 3
3, 4
5, 5
8, 9
"""

test_res = largest_area(parse_input(test_data))
assert test_res == 17

data = parse_input(Input(6).read())
largest_area(data)

3660

In [17]:
def largest_region(points, lim):
    max_x = max(p.x for p in points)
    max_y = max(p.y for p in points)
    
    region = []

    for x in range(max_x + 1):
        for y in range(max_y + 1):
            p2 = Point(x, y)
            region_sum = sum(
                manhattan_distance(p1, p2)
                for p1 in points
            )
            if region_sum < lim:
                region.append(p2)
    return len(region)

assert largest_region(parse_input(test_data), 32) == 16

data = parse_input(Input(6).read())
largest_region(data, 10000)

35928

## [Day 7: The Sum of Its Parts](https://adventofcode.com/2018/day/7)

This is an intersting tree type problem. I started with trying to build the tree in it's entirety and pruning nodes when required. This proved to be very error prone and note required, we can instead simply keep track of each node's parents or dependant node and process them once we've hit all the others.

By using a heap to store nodes that can be processed, we guarantee an O(1) complexity for accessing the next item.

In [18]:
def parse_input(data):
    data = re.findall('Step (\w) must be finished before step (\w) can begin.', data)
    res = defaultdict(set)
    for parent, child in data:
        if parent not in res:
            res[parent] = set()
        res[child].add(parent)
    return res


def lowest_order(data):
    to_visit = list(data.keys())
    heap = []
    visited = []
    while len(visited) != len(to_visit):
        can_visit = [
            key for key in to_visit
            if len(data[key]) == 0
        ]
        for key in can_visit:
            if key not in visited and key not in heap:
                heapq.heappush(heap, key)
        key = heapq.heappop(heap)
        visited.append(key)

        for key in data:
            data[key] = data[key] - set(visited)

    return cat(visited)
    

test_input = """
Step C must be finished before step A can begin.
Step C must be finished before step F can begin.
Step A must be finished before step B can begin.
Step A must be finished before step D can begin.
Step B must be finished before step E can begin.
Step D must be finished before step E can begin.
Step F must be finished before step E can begin.
"""

test_tree = parse_input(test_input)
assert lowest_order(test_tree) == 'CABDFE'

tree = parse_input(Input(7).read())
lowest_order(tree)

'JRHSBCKUTVWDQAIGYOPXMFNZEL'

Part two is trickier. Now we need to account for a series of workers performing tasks over time, with more tasks unlocking as others complete. The iteration is very similar to the previous one, but how we rank items that can be visted shifts to be base on the earliest time they can be worked on.

In [19]:
def key_val(key):
    # A = 1, B = 2 ...
    return ascii_uppercase.index(key) + 1


def lowest_build_time(data, num_workers, delay):
    original_data = data.copy()
    completion_times = {}

    worker_times = [0] * num_workers
    
    to_visit = list(data.keys())
    heap = []
    visited = []

    while len(visited) != len(to_visit):
        can_visit = [
            key for key in to_visit
            if len(data[key]) == 0
        ]
        for key in can_visit:
            time_reduction = max(worker_times)
            waiting_to_process = [h[1] for h in heap]
            if key not in visited and key not in waiting_to_process:
                next_free_worker = min(worker_times)
                worker_index = worker_times.index(next_free_worker)

                time_to_complete = delay + key_val(key)
                
                # Find the latest time when this node's parents will complete
                try:
                    parent_completion = max(
                        completion_times[p_key]
                        for p_key in original_data[key]
                    )
                except:
                    parent_completion = 0
                
                # Workers can be idle while waiting for parents to finish
                worker_starts = max(next_free_worker, parent_completion)
                completion_time = worker_starts + time_to_complete

                worker_times[worker_index] = completion_time
                completion_times[key] = completion_time
                
                # Add to the heap, sorting by the completion time
                heapq.heappush(heap, (completion_time,  key))
                
        # Pop next key to process
        _, key = heapq.heappop(heap)
        visited.append(key)

        for key in data:
            data[key] = data[key] - set(visited)
    return max(worker_times)


test_tree = parse_input(test_input)

assert lowest_build_time(test_tree, num_workers=2, delay=0) == 15

tree = parse_input(Input(7).read())
lowest_build_time(tree, num_workers=5, delay=60)

975

## [Day 8: Memory Maneuver](https://adventofcode.com/2018/day/8)

Another tree problem, although this one is actually best solved by creating the tree entirely.

The most difficult part of this however was parsing the input data in the first place!

In [20]:
class MemoryNode(Node):
    def __init__(self, name, meta):
        super().__init__(name)
        self.meta = meta
  
    @property
    def value(self):
        # This is essentially the solution to part two..
        val = 0
        if len(self.children) > 0:
            for index in self.meta:
                try:
                    child = self.children[index - 1]
                except IndexError:
                    continue
                val += child.value
        else:
            val = sum(self.meta)
        return val
        

In [21]:
from collections import deque


def parse_input(data):
    return list(map(int, data.split(' ')))


def build_nodes(data, nodes=None, parent=None):
    """
    Recursively consume the left side of the input stream,
    partially building the tree as we go.
    """
    if nodes is None:
        nodes = []
        
    num_children = data.popleft()
    num_meta_data = data.popleft()
    
    # wait for meta data until children have consumed
    # their data.
    node = MemoryNode(
        name='',
        meta=[]
    )
    
    for _ in range(num_children):
        build_nodes(data, nodes, parent=node)
    
    meta = []
    for i in range(num_meta_data):
        meta.append(data.popleft())
        
    node.meta = meta
    if parent:
        node.add_parent(parent)

    return node
    
    
def sum_meta_data(data):
    data = deque(data.copy())
    root = build_nodes(data)
    descenant_sum = sum(
        sum(node.meta) for node in root.decendants
    )
    return descenant_sum + sum(root.meta)


test_data = """2 3 0 3 10 11 12 1 1 0 1 99 2 1 1 2"""
assert sum_meta_data(parse_input(test_data)) == 138

sum_meta_data(
    parse_input(Input(8).read())
)

46781

Part two is easily solved by modifying our `MemoryNode` definition to be able to calculate it's value.

In [22]:
def root_value(data):
    data = deque(data.copy())
    root = build_nodes(data)
    return root.value
    

assert root_value(parse_input(test_data)) == 66
root_value(parse_input(Input(8).read()))

21405

## [Day 9: Marble Mania](https://adventofcode.com/2018/day/9)

The give away to this solution is that the marbles are placed in a _circle_. This immediately indicates we should use a doubly linked list, or in python land, a deque.

The ability to rotate the deque means that we're always performing O(1) operations, which helps a lot with the second part of the task.

In [23]:
def placing_marbles(num_players, last_marble_point):
    board = deque([0])
    scores = defaultdict(int)
    
    players = cycle(list(range(num_players)))
    current_player = next(players)

    for marble in range(1, last_marble_point + 1):
        if marble % 23 == 0:
            board.rotate(7)
            scores[current_player] += marble + board.pop()
            board.rotate(-1)
        else:
            board.rotate(-1)
            board.append(marble)
        current_player = next(players)        
    return max(scores.values())


assert placing_marbles(9, 25) == 32
assert placing_marbles(10, 1618) == 8317
assert placing_marbles(17, 1104) == 2764
assert placing_marbles(21, 6111) == 54718
assert placing_marbles(30, 5807) == 37305

placing_marbles(432, 71019)

400493

In [24]:
placing_marbles(432, 71019 * 100)

3338341690

## [Day 10: The Stars Align](https://adventofcode.com/2018/day/10)

This is a relatively simple problem, the only issue being you cannot write unit tests for it, and must rely on the user to validate it's correct.

In [25]:
Velocity = namedtuple('Velocity', 'x,y')
Star = namedtuple('Star', 'position,velocity')


def parse_input(data):
    matches = re.findall('position=<\s*(.+),\s*(.+)> velocity=<\s*(.+),\s*(.+)>', data)
    stars = []
    for x, y, v_x, v_y in matches:
        p = Point(int(x), int(y))
        v = Velocity(int(v_x), int(v_y))
        stars.append(
            Star(p, v)
        )
    return stars
        

In [26]:
test_data = """
position=< 9,  1> velocity=< 0,  2>
position=< 7,  0> velocity=<-1,  0>
position=< 3, -2> velocity=<-1,  1>
position=< 6, 10> velocity=<-2, -1>
position=< 2, -4> velocity=< 2,  2>
position=<-6, 10> velocity=< 2, -2>
position=< 1,  8> velocity=< 1, -1>
position=< 1,  7> velocity=< 1,  0>
position=<-3, 11> velocity=< 1, -2>
position=< 7,  6> velocity=<-1, -1>
position=<-2,  3> velocity=< 1,  0>
position=<-4,  3> velocity=< 2,  0>
position=<10, -3> velocity=<-1,  1>
position=< 5, 11> velocity=< 1, -2>
position=< 4,  7> velocity=< 0, -1>
position=< 8, -2> velocity=< 0,  1>
position=<15,  0> velocity=<-2,  0>
position=< 1,  6> velocity=< 1,  0>
position=< 8,  9> velocity=< 0, -1>
position=< 3,  3> velocity=<-1,  1>
position=< 0,  5> velocity=< 0, -1>
position=<-2,  2> velocity=< 2,  0>
position=< 5, -2> velocity=< 1,  2>
position=< 1,  4> velocity=< 2,  1>
position=<-2,  7> velocity=< 2, -2>
position=< 3,  6> velocity=<-1, -1>
position=< 5,  0> velocity=< 1,  0>
position=<-6,  0> velocity=< 2,  0>
position=< 5,  9> velocity=< 1, -2>
position=<14,  7> velocity=<-2,  0>
position=<-3,  6> velocity=< 2, -1>
"""

In [27]:
def is_message_complete(stars):
    # consider a message complete if every star
    # is at max 2 positions away from another
    good_stars = set()
    stars_to_test = set(stars)
    while stars_to_test:
        star = stars_to_test.pop()
        within_distance = False

        for other_star in stars:
            distance = manhattan_distance(star.position, other_star.position)
            if distance in (1, 2):
                good_stars.add(star)
                good_stars.add(other_star)
                within_distance = True

        if not within_distance:
            # No point testing other stars
            return False
        
        stars_to_test = stars_to_test - good_stars
    return True

def print_message(stars):
    """Output stars to the stdout."""
    x_max = max(star.position.x for star in stars)
    y_max = max(star.position.y for star in stars)
    
    x_min = min(star.position.x for star in stars)
    y_min = min(star.position.y for star in stars)
    
    positions = set(star.position for star in stars)
        
    for y in range(y_min, y_max + 1):
        line = []
        for x in range(x_min, x_max + 1):
            pos = (x, y)
            char = '#' if pos in positions else '.'
            line.append(char)
        print(cat(line))

        
def advance_time(stars):
    new_stars = []
    for star in stars:
        new_stars.append(
            Star(
                Point(
                    star.position.x + star.velocity.x,
                    star.position.y + star.velocity.y
                ),
                star.velocity
            )
        )
    return new_stars


def find_message(stars):
    time = 0
    while True:
        stars = advance_time(stars)
        time += 1

        if is_message_complete(stars):
            print_message(stars)
            return time
        

find_message(parse_input(test_data))
find_message(parse_input(Input(10).read()))

#...#..###
#...#...#.
#...#...#.
#####...#.
#...#...#.
#...#...#.
#...#...#.
#...#..###
######..#....#....##....######..#####...######..#....#..#####.
#.......#....#...#..#........#..#....#.......#..#....#..#....#
#.......#....#..#....#.......#..#....#.......#..#....#..#....#
#.......#....#..#....#......#...#....#......#...#....#..#....#
#####...######..#....#.....#....#####......#....######..#####.
#.......#....#..######....#.....#.........#.....#....#..#.....
#.......#....#..#....#...#......#........#......#....#..#.....
#.......#....#..#....#..#.......#.......#.......#....#..#.....
#.......#....#..#....#..#.......#.......#.......#....#..#.....
######..#....#..#....#..######..#.......######..#....#..#.....


10136

## [Day 11: Chronal Charge](https://adventofcode.com/2018/day/11)

This was an interesting problem that required the precomputation of sums for a given cell - after doing it the niave way and seeing that it simply would not work.

If we have the grid:

```
-2  -4   4   4   
-4   4   4   4 
 4   3   3   4 
 1   1   2   4
```

By creating a cache of values that have points (x, y) which represent the sum of all points (i, j) for i <= x and j <= y, we have the following:

```
-2  -6  -2   2   
-6  -6   2  10 
-2   1  12  24 
-1   3  16  32
```

The point (x, y) in this grid can be writen in terms of its neighbours, as they will have already been calculated: (x - 1, y) + (x, y - 1) - (x - 1, y - 1). Essentially this just says, give me the sum of all previous x and y points, removing any duplicates. From here we can calculate any rectangle on the grid by taking the point (x, y) and removing the parts we do not are about.

In [28]:
def cell_power_level(grid_id, x, y):
    rack_id = x + 10
    power_level = rack_id * y
    power_level += grid_id
    power_level *= rack_id

    hundreds_digit = (abs(power_level) % 1000 ) // 100
    return hundreds_digit - 5

assert cell_power_level(57, 122, 79) == -5
assert cell_power_level(39, 217, 196) == 0
assert cell_power_level(71, 101, 153) == 4

In [29]:
def get_power_levels(grid_id):
    power_levels = {}
    for x in range(1, 301):
        for y in range(1, 301):
            point = Point(x, y)
            power_levels[point] = cell_power_level(grid_id, x, y)
    return power_levels


def get_power_sum(grid_id):
    # Build a cumulative / prefix sum of all points, where (x, y) is the
    # sum of all values of (i, j) for i <= x and j <= y.
    power_levels = get_power_levels(grid_id)
    power_sum = {}
    for x in range(1, 301):
        for y in range(1, 301):
            power_sum[(x, y)] = (
                power_levels[(x, y)] +
                power_sum.get((x - 1, y), 0) +
                power_sum.get((x, y - 1), 0) -
                power_sum.get((x - 1, y - 1), 0) 
            )
    return power_sum


def largest_total_power(grid_id, power_sum, size=3):
    max_power = None
    winning_cell = None
    
    point_offset = size - 1
    
    x_min = size - 1
    x_max = 301 - size - 1
    for x in range(x_min, x_max):
        for y in range(x_min, x_max):
            power = (
                power_sum[(x + size, y + size)] -
                power_sum[(x, y + size)] -
                power_sum[(x + size, y)] +
                power_sum[(x, y)]
            )
            if max_power is None or power > max_power:
                max_power = power
                # Top left node
                winning_cell = (x + 1, y + 1)
    return max_power, winning_cell


def largest_total_power_for_grid(grid_id):
    power_sum = get_power_sum(grid_id)
    return largest_total_power(grid_id, power_sum)

assert largest_total_power_for_grid(18) == (29, (33, 45))
assert largest_total_power_for_grid(42) == (30, (21, 61))
largest_total_power_for_grid(5719)

(29, (21, 34))

In [30]:
def largest_total_power_for_size(grid_id):
    power_sum = get_power_sum(grid_id)
    max_power = None
    winning_power = None
    winning_cell = None
    for size in range(3, 300):
        power, cell = largest_total_power(grid_id, power_sum, size)
        if max_power is None or (power is not None and power > max_power):
            max_power = power
            winning_cell = cell
            winning_size = size
    return max_power, winning_cell, winning_size

assert largest_total_power_for_size(18) == (113, (90, 269), 16)
assert largest_total_power_for_size(42) == (119, (232, 251), 12)
largest_total_power_for_size(5719)

(124, (90, 244), 16)